In [75]:
import pandas as pd
from preprocessing_stock import Stock_Preprocessing
from calculation import CalcIntersection

In [84]:
stock_id = 'NVDA'

stock_processor = Stock_Preprocessing()
calc = CalcIntersection()

# load data from Yahoo
data = stock_processor.load_data([stock_id])
# Save data to csv
stock_processor.save_dataframe_to_file(data=data, file_path=f'data/{stock_id}.csv')


2020-05-26 09:46:23.990 INFO:	Init class Stock_Preprocessing ...
2020-05-26 09:46:23.992 INFO:	... Init class Stock_Preprocessing done
2020-05-26 09:46:23.993 INFO:	load_data(stock_ids=['NVDA']) ...
2020-05-26 09:46:23.994 INFO:	download_stock_data(stock_ids=['NVDA']) ...
2020-05-26 09:46:24.022 DEBUG:	Starting new HTTPS connection (1): query1.finance.yahoo.com:443
2020-05-26 09:46:24.162 DEBUG:	https://query1.finance.yahoo.com:443 "GET /v8/finance/chart/NVDA?period1=-2208988800&period2=1590457584&interval=1d&includePrePost=False&events=div%2Csplits HTTP/1.1" 200 None


[*********************100%***********************]  1 of 1 completed

2020-05-26 09:46:24.316 INFO:	... download_stock_data()
2020-05-26 09:46:24.317 INFO:	... load_data()
2020-05-26 09:46:24.319 INFO:	save_dataframe_to_file(), DataFrame will be saved in data/NVDA.csv ...
2020-05-26 09:46:24.395 INFO:	... save_dataframe_to_file()


In [85]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-05-18,350.420013,356.660004,347.220001,350.010010,350.010010,19410100.0
2020-05-19,351.609985,363.500000,350.510010,352.220001,352.220001,17933000.0
2020-05-20,359.640015,361.720001,355.549988,358.799988,358.799988,14688400.0
2020-05-21,362.059998,362.100006,349.500000,351.010010,351.010010,19026700.0
2020-05-22,353.010010,363.720001,348.529999,361.049988,361.049988,25969100.0


In [41]:
# Calculate MAs (e.g. MA3, MA5, MA10 ...)
days = [3, 5, 10, 20, 50, 100]
mas = stock_processor.calculate_mas(data=data['Close'], days=days, return_type='list')

In [42]:
type(mas)

list

In [44]:
len(mas)

6

In [45]:
type(mas[0])

pandas.core.series.Series

In [46]:
mas[0].tail()

Date
2020-05-18    336.953339
2020-05-19    347.286672
2020-05-20    353.676666
2020-05-21    354.010000
2020-05-22    356.953328
Name: Close, dtype: float64

In [47]:
data['ma3'] = mas[0]

In [49]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume,ma3
Date,,,,,,,
2020-05-18,350.420013,356.660004,347.220001,350.010010,350.010010,19410100.0,336.953339
2020-05-19,351.609985,363.500000,350.510010,352.220001,352.220001,17933000.0,347.286672
2020-05-20,359.640015,361.720001,355.549988,358.799988,358.799988,14688400.0,353.676666
2020-05-21,362.059998,362.100006,349.500000,351.010010,351.010010,19026700.0,354.010000
2020-05-22,353.010010,363.720001,348.529999,361.049988,361.049988,25951100.0,356.953328


In [78]:
# Calculate MAs (e.g. MA3, MA5, MA10 ...)
days = [3, 5, 10, 20, 50, 100]
mas = stock_processor.calculate_mas(data=data['Close'], days=days, return_type='dict')

In [79]:
# for k in mas:
    
data[f'ma{k}'] = mas[k]

In [80]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume,ma100
Date,,,,,,,
2020-05-18,350.420013,356.660004,347.220001,350.010010,350.010010,19410100.0,263.859301
2020-05-19,351.609985,363.500000,350.510010,352.220001,352.220001,17933000.0,264.995301
2020-05-20,359.640015,361.720001,355.549988,358.799988,358.799988,14688400.0,266.191401
2020-05-21,362.059998,362.100006,349.500000,351.010010,351.010010,19026700.0,267.332801
2020-05-22,353.010010,363.720001,348.529999,361.049988,361.049988,25969100.0,268.620101


In [81]:
def merge_mas_into_data(mas:'dict[pd.DataFrame]', data=pd.DataFrame):
    ''' merge calculated mas (list of df) into data (df)
        params: (list)(DataFrame) @mas
        return: (DataFrame) merged stock data
    '''
    for day in mas:
        data[f'ma{day}'] = mas[day]
        
    return data

In [82]:
new_data = merge_mas_into_data(mas=mas, data=data)

In [83]:
data.tail()

,Open,High,Low,Close,Adj Close,Volume,ma100,ma3,ma5,ma10,ma20,ma50
Date,,,,,,,,,,,,
2020-05-18,350.420013,356.660004,347.220001,350.010010,350.010010,19410100.0,263.859301,336.953339,326.832007,316.568002,302.409500,270.973600
2020-05-19,351.609985,363.500000,350.510010,352.220001,352.220001,17933000.0,264.995301,347.286672,334.856006,322.416003,306.545000,273.109200
2020-05-20,359.640015,361.720001,355.549988,358.799988,358.799988,14688400.0,266.191401,353.676666,344.376001,328.517001,310.177499,275.063600
2020-05-21,362.059998,362.100006,349.500000,351.010010,351.010010,19026700.0,267.332801,354.010000,350.334003,333.131003,313.527499,277.154401
2020-05-22,353.010010,363.720001,348.529999,361.049988,361.049988,25969100.0,268.620101,356.953328,354.617999,337.986002,317.100499,280.049200


In [86]:
def calculate_mas_crossing(days:'list[int]', data=pd.DataFrame)->'pd.DataFrame':
    ''' calculate mas crossing signal, e.g. ma3 x ma5, ma3 x ma10 ...
        params: (list) @days, e.g. days = [3, 5, 10]
        return: (pd.DataFrame) mas crossing signals, e.g. ['3x5']=1, ['3x10']=0 ...
    '''
    crossing_mas = {} # e.g. crossing_mas[3x5], crossing_mas[3x10] ...
    for day in mas:
        for cross_day in mas:
            if day != cross_day:
                crossing_mas[f'{day}x{cross_day}'] = calc.cross_signal(data, day, cross_day)
    return crossing_mas

In [87]:
crossing_mas = calculate_mas_crossing(mas=mas, data=data)

2020-05-26 09:49:11.588 DEBUG:	                 Close       Close
Date                              
2020-05-18  336.953339  326.832007
2020-05-19  347.286672  334.856006
2020-05-20  353.676666  344.376001
2020-05-21  354.010000  350.334003
2020-05-22  356.953328  354.617999
2020-05-26 09:49:11.599 DEBUG:	                 Close       Close
Date                              
2020-05-18  336.953339  316.568002
2020-05-19  347.286672  322.416003
2020-05-20  353.676666  328.517001
2020-05-21  354.010000  333.131003
2020-05-22  356.953328  337.986002
2020-05-26 09:49:11.609 DEBUG:	                 Close       Close
Date                              
2020-05-18  336.953339  302.409500
2020-05-19  347.286672  306.545000
2020-05-20  353.676666  310.177499
2020-05-21  354.010000  313.527499
2020-05-22  356.953328  317.100499
2020-05-26 09:49:11.618 DEBUG:	                 Close       Close
Date                              
2020-05-18  336.953339  270.973600
2020-05-19  347.286672  273.109200
2

In [88]:
type(crossing_mas)

dict

In [89]:
len(crossing_mas)

30

In [90]:
for k in crossing_mas:
    print(k)

3x5
3x10
3x20
3x50
3x100
5x3
5x10
5x20
5x50
5x100
10x3
10x5
10x20
10x50
10x100
20x3
20x5
20x10
20x50
20x100
50x3
50x5
50x10
50x20
50x100
100x3
100x5
100x10
100x20
100x50


In [91]:
crossing_mas[k]

0